# ElGamal Encryption

ElGamal is a public-key encryption scheme based on Diffie–Hellman. It is **probabilistic**, meaning that encrypting the same message twice produces different ciphertexts for enhanced privacy.

## Overview

- **Public Parameters:**  
  - **Prime (p):** A large prime number.  
  - **Generator (g):** A generator of the multiplicative group modulo p.

- **Key Generation (Receiver):**  
  - **Private Key (x):** A random number in the range 1 to p-2.  
  - **Public Key (y):** Computed as `y = g^x mod p`.

- **Encryption (Sender, for message m):**  
  1. Choose a fresh random number **k** (never reuse it).  
  2. Compute `c1 = g^k mod p`.  
  3. Compute `s = y^k mod p`.  
  4. Compute `c2 = m * s mod p`.  
  5. The ciphertext is the pair `(c1, c2)`.

- **Decryption (Receiver):**  
  1. Compute `s = c1^x mod p`.  
  2. Recover the message using the modular inverse of s:  
     `m = c2 * s⁻¹ mod p`.

## Parameters Summary

- **p:** A large prime (public).  
- **g:** A generator modulo p (public).  
- **x:** The private key (secret).  
- **y = g^x:** The public key (public).  
- **k:** A fresh random per-encryption (must never be reused).

## Security Notes

- **Fresh k:** Reusing k across encryptions can lead to exposure of the private key.
- **Message Encoding:** The message m must be an integer in the range 1 to p-1. For arbitrary data, use appropriate encoding, padding, and consider a hybrid encryption scheme.
- **Malleability:** ElGamal does not provide integrity. To safeguard against tampering, combine with a Message Authentication Code (MAC) or use an Authenticated Encryption with Associated Data (AEAD) scheme.
- **Production Tip:** For practical systems, prefer elliptic-curve variants (EC ElGamal) or hybrid approaches available in standard cryptographic libraries.

In [1]:
import secrets

# --- Public parameters (tiny, INSECURE example) ---
# In real life, p must be very large (e.g., 2048+ bits)
p = 467      # prime
g = 2        # generator modulo p

# --- Key generation (receiver) ---
x = secrets.randbelow(p - 2) + 1      # private key in [1, p-2]
y = pow(g, x, p)                      # public key

print("Public key (p, g, y):", (p, g, y))
print("Private key x:", x)

# --- Helper: modular inverse using Fermat's little theorem (since p is prime) ---
def inv_mod(a, p):
    # returns a^{-1} mod p
    return pow(a, p - 2, p)

# --- Encrypt (sender) ---
# Message as an integer 1..p-1. For demo, just pick a small int < p
m = 123
assert 1 <= m < p

k = secrets.randbelow(p - 2) + 1      # fresh random per encryption
c1 = pow(g, k, p)
s  = pow(y, k, p)                      # shared mask
c2 = (m * s) % p
ciphertext = (c1, c2)

print("Ciphertext (c1, c2):", ciphertext)

# --- Decrypt (receiver) ---
c1, c2 = ciphertext
s = pow(c1, x, p)
m_recovered = (c2 * inv_mod(s, p)) % p

print("Recovered message:", m_recovered)
assert m_recovered == m


Public key (p, g, y): (467, 2, 460)
Private key x: 381
Ciphertext (c1, c2): (168, 290)
Recovered message: 123
